In [118]:
import pandas as pd
import numpy as np

In [119]:
# leer el archivo de excel
df = pd.read_excel('superstore.xlsx')

In [120]:
print(df.head())

   Row ID        Order ID Order Date  Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
1       2  CA-2016-152156 2016-11-08 2016-11-11    Second Class    CG-12520   
2       3  CA-2016-138688 2016-06-12 2016-06-16    Second Class    DV-13045   
3       4  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   
4       5  US-2015-108966 2015-10-11 2015-10-18  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   South  FUR-BO-10

In [121]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

 - Transformar toda la columna Order Date a datetime

In [128]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

<h1>Recency </h1>

In [129]:
# Calcula la fecha máxima de order Date
last_order_date = df['Order Date'].max()
last_order_date


Timestamp('2017-12-30 00:00:00')

<h1>Última orden por cada cliente </h1>

In [127]:
last_order = df.groupby('Customer ID')['Order Date'].max()
last_order

Customer ID
AA-10315   2017-06-29
AA-10375   2017-12-11
AA-10480   2017-04-15
AA-10645   2017-11-05
AB-10015   2016-11-10
              ...    
XP-21865   2017-11-17
YC-21895   2017-12-26
YS-21880   2017-12-21
ZC-21910   2017-11-06
ZD-21925   2017-06-11
Name: Order Date, Length: 793, dtype: datetime64[ns]

<h1>Resta entre fechas </h1>

In [135]:
resta = last_order_date - last_order
resta

Customer ID
AA-10315   184 days
AA-10375    19 days
AA-10480   259 days
AA-10645    55 days
AB-10015   415 days
             ...   
XP-21865    43 days
YC-21895     4 days
YS-21880     9 days
ZC-21910    54 days
ZD-21925   202 days
Name: Order Date, Length: 793, dtype: timedelta64[ns]

<h1>Monetary</h1>

In [133]:
how_much_sales = df.groupby('Customer ID')['Sales'].sum()

In [134]:
how_much_sales.head()

Customer ID
AA-10315    5563.560
AA-10375    1056.390
AA-10480    1790.512
AA-10645    5086.935
AB-10015     886.156
Name: Sales, dtype: float64

<h1>Frecuency </h1>

In [136]:
quantity = df.groupby('Customer ID')['Quantity'].sum()
quantity.head()

Customer ID
AA-10315    30
AA-10375    41
AA-10480    36
AA-10645    64
AB-10015    13
Name: Quantity, dtype: int64

In [137]:
new_df = pd.concat([resta, how_much_sales, quantity], axis=1)
#df['frequency'] = df[quantity]
print(new_df)

            Order Date     Sales  Quantity
Customer ID                               
AA-10315      184 days  5563.560        30
AA-10375       19 days  1056.390        41
AA-10480      259 days  1790.512        36
AA-10645       55 days  5086.935        64
AB-10015      415 days   886.156        13
...                ...       ...       ...
XP-21865       43 days  2374.658       100
YC-21895        4 days  5454.350        31
YS-21880        9 days  6720.444        58
ZC-21910       54 days  8025.707       105
ZD-21925      202 days  1493.944        32

[793 rows x 3 columns]


In [139]:
# transform timedelta to int
new_df['Order Date'] = new_df['Order Date'].dt.days
new_df.head()

,Order Date,Sales,Quantity
Customer ID,,,
AA-10315,184,5563.560,30
AA-10375,19,1056.390,41
AA-10480,259,1790.512,36
AA-10645,55,5086.935,64
AB-10015,415,886.156,13


In [141]:
new_df.dtypes

Order Date      int64
Sales         float64
Quantity        int64
dtype: object